In [1]:
# Option 2: Re-Identification in a Single Feed - Initial Setup

In [ ]:
# Install YOLOv8 and dependencies for tracking
!pip install -U ultralytics opencv-python-headless lap

# Optional: For plotting, progress bars, and basic ML utils
!pip install -q matplotlib pandas tqdm scikit-learn
!pip install git+https://github.com/facebookresearch/segment-anything.git

print("\n✅ All packages installed. If this is your first run, go to Runtime → Restart runtime NOW ⚠️")


In [3]:
# Step 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("Google Drive Mounted Successfully!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive Mounted Successfully!


**Step 3: Run Ultra-Accurate Player Tracking**
This step performs robust player tracking using a custom tracking pipeline built on top of YOLOv8 detections. It includes:

🔧 What Happens in This Step:


*   Load the trained YOLOv8 model (best.pt) from Google Drive.


*   Read the input match video (15sec_input_720p.mp4) from Google Drive.


*   Use YOLOv8 with ByteTrack to get detections and temporary tracker IDs.


*   Apply a custom Re-Identification (Re-ID) pipeline that assigns consistent Player IDs using:


1.   Deep Features extracted from ResNet50

2.   Color Histograms

3.   Dominant Colors

4.   Bounding Box IoU

5.   Motion Prediction



*   Draw bounding boxes and persistent player IDs on each frame with visual   confidence levels.

*   Save:

📹 A fully annotated output video (ultra_accurate_tracking.avi)

🖼️ Each annotated frame as images inside /Outputs/tracked_frames/

In [ ]:
# ==========================================
# ULTRA-ACCURATE PLAYER TRACKING SYSTEM
# ==========================================
# This system combines YOLO object detection with advanced tracking techniques
# to maintain consistent player identities across video frames with minimal ID switching

from ultralytics import YOLO
import cv2
import os
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
import torch.nn as nn
from collections import defaultdict, deque
from scipy.spatial.distance import cosine
from sklearn.cluster import DBSCAN
import math

# ==========================================
# FILE PATHS AND SETUP
# ==========================================
# Define all necessary paths for model, input video, and output directory
model_path = "/content/drive/MyDrive/Player_Tracking/Models/best.pt"
video_path = "/content/drive/MyDrive/Player_Tracking/Input_Video/15sec_input_720p.mp4"
output_dir = "/content/drive/MyDrive/Player_Tracking/Outputs"
os.makedirs(output_dir, exist_ok=True)

# ==========================================
# YOLO MODEL INITIALIZATION
# ==========================================
# Load the pre-trained YOLO model for player detection
# Using try-except for robust error handling during model loading
try:
    model = YOLO(model_path)
    print(f"✅ Model loaded from: {model_path}")
except Exception as e:
    print(f"❌ Failed to load model: {e}")
    exit()

# ==========================================
# ENHANCED FEATURE EXTRACTOR CLASS
# ==========================================
# This class uses ResNet50 for deep feature extraction to create robust player representations
# ResNet50 is chosen because it provides rich visual features that are excellent for person re-identification
class EnhancedFeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        # Use pre-trained ResNet50 as backbone for feature extraction
        # Pre-trained weights help with better feature representation
        self.backbone = resnet50(pretrained=True)
        # Replace final classification layer with a feature vector of size 256
        # This creates a compact but informative representation for each player
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, 256)
        self.backbone.eval()  # Set to evaluation mode for inference

    def forward(self, x):
        return self.backbone(x)

# ==========================================
# DEVICE SETUP AND MODEL INITIALIZATION
# ==========================================
# Use GPU if available for faster feature extraction, fallback to CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
feature_extractor = EnhancedFeatureExtractor().to(device)
feature_extractor.eval()  # Ensure model is in evaluation mode

# ==========================================
# IMAGE PREPROCESSING PIPELINE
# ==========================================
# Standard ImageNet preprocessing pipeline for ResNet50
# This ensures input images are in the format expected by the pre-trained model
preprocess = transforms.Compose([
    transforms.ToPILImage(),           # Convert numpy array to PIL Image
    transforms.Resize((224, 224)),     # Resize to standard input size for ResNet50
    transforms.ToTensor(),             # Convert to tensor format
    transforms.Normalize(              # Normalize with ImageNet statistics
        mean=[0.485, 0.456, 0.406],   # ImageNet mean values
        std=[0.229, 0.224, 0.225]     # ImageNet standard deviation values
    )
])

# ==========================================
# VIDEO INPUT/OUTPUT SETUP
# ==========================================
# Open input video and configure output video writer
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"❌ Error opening video file: {video_path}")
    exit()

# Get video properties and set output to 30fps for consistency
original_fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
output_video_path = os.path.join(output_dir, "ultra_accurate_tracking.avi")
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # XVID codec for good compression
out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (width, height))  # Set to 30fps

# Create directory for individual frames (useful for debugging and analysis)
frames_dir = os.path.join(output_dir, "tracked_frames")
os.makedirs(frames_dir, exist_ok=True)

# ==========================================
# ULTRA-ACCURATE PLAYER TRACKER CLASS
# ==========================================
# This is the core tracking system that maintains player identities across frames
# It combines multiple techniques for maximum accuracy and minimal ID switching
class UltraAccuratePlayerTracker:
    def __init__(self, feature_extractor, device):
        self.feature_extractor = feature_extractor
        self.device = device

        # ==========================================
        # CORE TRACKING DATA STRUCTURES
        # ==========================================
        # These dictionaries maintain all player information across frames
        self.tracker_to_player = {}     # Maps YOLO tracker IDs to our player IDs
        self.player_features = {}       # Stores feature history for each player
        self.player_boxes = {}          # Stores bounding box history for each player
        self.player_positions = {}      # Stores center position history for each player
        self.player_confidence = {}     # Confidence scores for each player assignment

        # ==========================================
        # SYSTEM CONFIGURATION
        # ==========================================
        # These parameters control the tracking behavior and accuracy
        self.next_player_id = 1         # Counter for assigning new player IDs
        self.max_players = 22           # Maximum expected players (soccer context)
        self.feature_history_size = 400  # How many feature vectors to remember per player
        self.position_history_size = 400 # How many positions to remember per player
        self.box_history_size = 400     # How many bounding boxes to remember per player

        # ==========================================
        # MATCHING THRESHOLDS
        # ==========================================
        # These thresholds determine when to match detections to existing players
        self.feature_similarity_threshold = 0.75  # Minimum feature similarity for matching
        self.iou_threshold = 0.5                  # Minimum IoU for spatial matching
        self.position_threshold = 100             # Maximum pixel distance for position matching
        self.confidence_threshold = 0.6           # Minimum confidence for ID assignment

        # ==========================================
        # MISSING TRACKER MANAGEMENT
        # ==========================================
        # Handle cases where players temporarily disappear from detection
        self.missing_trackers = {}      # Track how long each tracker has been missing
        self.max_missing_frames = 20    # Maximum frames before removing a tracker

        # ==========================================
        # ID SWITCHING PREVENTION
        # ==========================================
        # Prevent rapid ID changes that can cause instability
        self.id_switch_cooldown = {}    # Cooldown period after ID assignment
        self.cooldown_frames = 10       # Frames to wait before allowing ID changes

        # ==========================================
        # MOTION PREDICTION
        # ==========================================
        # Store velocity information for predicting player movement
        self.player_velocities = {}     # Velocity vectors for each player

    def is_player_detection(self, box, class_id=None):
        """
        PLAYER DETECTION FILTER
        =====================
        Filters out non-player detections (referees, balls, etc.) using heuristics
        This is crucial for maintaining accuracy by only tracking actual players
        """
        # Add your class filtering logic here
        # If your model outputs class IDs, filter by player class
        # For now, we'll use box size and position heuristics

        x1, y1, x2, y2 = box
        box_width = x2 - x1
        box_height = y2 - y1
        aspect_ratio = box_width / max(box_height, 1)

        # Filter out very small detections (likely ball or false positives)
        if box_width < 20 or box_height < 30:
            return False

        # Filter out very wide detections (likely not a person)
        # Players should have a relatively normal human aspect ratio
        if aspect_ratio > 2.0:
            return False

        # Filter out very tall detections (likely referee or wrong detection)
        # Extremely tall/thin detections are usually errors
        if aspect_ratio < 0.3:
            return False

        return True

    def extract_multi_features(self, frame, box):
        """
        MULTI-MODAL FEATURE EXTRACTION
        =============================
        Extracts three types of features for robust player identification:
        1. Deep features (ResNet50) - Most important for person re-identification
        2. Color histogram - Captures uniform colors and patterns
        3. Dominant color - Simple but effective for jersey identification
        """
        x1, y1, x2, y2 = box

        # Ensure coordinates are within frame bounds to prevent errors
        x1 = max(0, min(x1, frame.shape[1]-1))
        y1 = max(0, min(y1, frame.shape[0]-1))
        x2 = max(x1+1, min(x2, frame.shape[1]))
        y2 = max(y1+1, min(y2, frame.shape[0]))

        roi = frame[y1:y2, x1:x2]  # Extract region of interest
        if roi.size == 0:
            return np.zeros(256), np.zeros(64), np.zeros(3)

        # ==========================================
        # 1. DEEP FEATURES USING RESNET50
        # ==========================================
        # These features capture complex visual patterns and are most important for identification
        deep_features = np.zeros(256)
        try:
            roi_tensor = preprocess(roi).unsqueeze(0).to(self.device)
            with torch.no_grad():  # Disable gradient computation for efficiency
                deep_features = self.feature_extractor(roi_tensor).cpu().numpy().flatten()
        except Exception as e:
            print(f"Deep feature extraction error: {e}")

        # ==========================================
        # 2. COLOR HISTOGRAM FEATURES
        # ==========================================
        # Capture color distribution which is useful for jersey identification
        color_features = np.zeros(64)
        try:
            roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)  # HSV is better for color analysis
            # Create 2D histogram for hue and saturation (8x8 bins)
            hist = cv2.calcHist([roi_hsv], [0, 1], None, [8, 8], [0, 180, 0, 256])
            color_features = hist.flatten() / (hist.sum() + 1e-7)  # Normalize histogram
        except Exception as e:
            print(f"Color feature extraction error: {e}")

        # ==========================================
        # 3. DOMINANT COLOR FEATURES
        # ==========================================
        # Simple but effective feature for jersey color identification
        dominant_color = np.zeros(3)
        try:
            roi_resized = cv2.resize(roi, (64, 64))  # Resize for consistent processing
            roi_flat = roi_resized.reshape(-1, 3)   # Flatten to get all pixels
            dominant_color = np.mean(roi_flat, axis=0)  # Average color
        except Exception as e:
            print(f"Dominant color extraction error: {e}")

        return deep_features, color_features, dominant_color

    def compute_comprehensive_similarity(self, features1, features2):
        """
        COMPREHENSIVE SIMILARITY COMPUTATION
        ==================================
        Combines multiple feature types with appropriate weighting
        Deep features are weighted most heavily as they're most discriminative
        """
        deep1, color1, dom1 = features1
        deep2, color2, dom2 = features2

        # ==========================================
        # DEEP FEATURE SIMILARITY (MOST IMPORTANT)
        # ==========================================
        # Cosine similarity works well for normalized deep features
        deep_sim = 0.0
        if np.linalg.norm(deep1) > 0 and np.linalg.norm(deep2) > 0:
            deep_sim = 1 - cosine(deep1, deep2)

        # ==========================================
        # COLOR HISTOGRAM SIMILARITY
        # ==========================================
        # Captures jersey color patterns
        color_sim = 0.0
        if np.linalg.norm(color1) > 0 and np.linalg.norm(color2) > 0:
            color_sim = 1 - cosine(color1, color2)

        # ==========================================
        # DOMINANT COLOR SIMILARITY
        # ==========================================
        # Simple color matching for jersey identification
        dom_sim = 0.0
        if np.linalg.norm(dom1) > 0 and np.linalg.norm(dom2) > 0:
            dom_sim = 1 - np.linalg.norm(dom1 - dom2) / (np.linalg.norm(dom1) + np.linalg.norm(dom2))

        # ==========================================
        # WEIGHTED COMBINATION
        # ==========================================
        # Deep features get highest weight (60%), color histogram (30%), dominant color (10%)
        combined_sim = 0.6 * deep_sim + 0.3 * color_sim + 0.1 * dom_sim
        return combined_sim, deep_sim, color_sim, dom_sim

    def compute_iou(self, boxA, boxB):
        """
        INTERSECTION OVER UNION (IoU) COMPUTATION
        =======================================
        Measures spatial overlap between bounding boxes
        Essential for tracking continuity - players shouldn't jump around
        """
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])

        # Compute intersection area
        interArea = max(0, xB - xA) * max(0, yB - yA)
        if interArea == 0:
            return 0.0

        # Compute union area
        boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
        boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

        # IoU = intersection / union
        return interArea / float(boxAArea + boxBArea - interArea)

    def predict_position(self, player_id):
        """
        MOTION PREDICTION
        ================
        Predicts where a player will be in the next frame based on velocity
        Helps with tracking continuity when detections are temporarily lost
        """
        if player_id not in self.player_positions or len(self.player_positions[player_id]) < 2:
            return None

        positions = list(self.player_positions[player_id])
        if len(positions) < 2:
            return positions[-1]

        # Calculate velocity from last two positions
        last_pos = positions[-1]
        prev_pos = positions[-2]
        velocity = (last_pos[0] - prev_pos[0], last_pos[1] - prev_pos[1])

        # Predict next position using linear motion model
        predicted_pos = (last_pos[0] + velocity[0], last_pos[1] + velocity[1])
        return predicted_pos

    def get_position_distance(self, box, predicted_pos):
        """
        POSITION DISTANCE CALCULATION
        ============================
        Calculates distance between detection and predicted position
        Used to maintain tracking continuity
        """
        if predicted_pos is None:
            return float('inf')

        box_center = ((box[0] + box[2]) / 2, (box[1] + box[3]) / 2)
        distance = math.sqrt((box_center[0] - predicted_pos[0])**2 +
                           (box_center[1] - predicted_pos[1])**2)
        return distance

    def assign_player_id(self, tracker_id, box, features, frame_count):
        """
        CORE PLAYER ID ASSIGNMENT LOGIC
        ==============================
        This is the heart of the tracking system. It decides which player ID
        to assign to each detection using multiple criteria for maximum accuracy
        """

        # ==========================================
        # CHECK EXISTING TRACKER ASSIGNMENT
        # ==========================================
        # If tracker already has a player ID, verify it's still valid
        if tracker_id in self.tracker_to_player:
            player_id = self.tracker_to_player[tracker_id]

            # Check if this ID is in cooldown (prevent rapid switching)
            if player_id in self.id_switch_cooldown:
                if frame_count - self.id_switch_cooldown[player_id] < self.cooldown_frames:
                    # Update data and return existing ID
                    self.update_player_data(player_id, box, features)
                    return player_id
                else:
                    # Cooldown expired, remove it
                    del self.id_switch_cooldown[player_id]

            # Update player data and return existing assignment
            self.update_player_data(player_id, box, features)
            return player_id

        # ==========================================
        # MATCH WITH EXISTING PLAYERS
        # ==========================================
        # Try to match this detection with existing players using multiple criteria
        best_matches = []

        for player_id in self.player_features.keys():
            # Skip if player is currently assigned to another active tracker
            if player_id in self.tracker_to_player.values():
                continue

            # Get average features for comparison
            if not self.player_features[player_id]:
                continue

            # Use recent features for comparison (last 20 feature vectors)
            recent_features = list(self.player_features[player_id])[-20:]

            # ==========================================
            # COMPUTE FEATURE SIMILARITIES
            # ==========================================
            # Compare with multiple recent features to get best match
            similarities = []
            for stored_features in recent_features:
                sim, deep_sim, color_sim, dom_sim = self.compute_comprehensive_similarity(
                    features, stored_features)
                similarities.append((sim, deep_sim, color_sim, dom_sim))

            # Use best similarity from recent history
            best_sim = max(similarities, key=lambda x: x[0])
            combined_sim, deep_sim, color_sim, dom_sim = best_sim

            # ==========================================
            # COMPUTE SPATIAL OVERLAP (IoU)
            # ==========================================
            # Check if detection overlaps with recent player positions
            iou = 0.0
            if player_id in self.player_boxes and self.player_boxes[player_id]:
                recent_boxes = list(self.player_boxes[player_id])[-5:]
                ious = [self.compute_iou(box, stored_box) for stored_box in recent_boxes]
                iou = max(ious)

            # ==========================================
            # POSITION PREDICTION SCORING
            # ==========================================
            # Score based on how close detection is to predicted position
            predicted_pos = self.predict_position(player_id)
            position_distance = self.get_position_distance(box, predicted_pos)
            position_score = max(0, 1 - position_distance / self.position_threshold)

            # ==========================================
            # COMBINED SCORING
            # ==========================================
            # Combine all factors with appropriate weights
            final_score = (0.5 * combined_sim +      # Feature similarity (most important)
                          0.2 * iou +                # Spatial continuity
                          0.2 * position_score +     # Motion prediction
                          0.1 * self.player_confidence.get(player_id, 0))  # Historical confidence

            # ==========================================
            # QUALITY THRESHOLDS
            # ==========================================
            # Only consider matches that meet minimum quality requirements
            if (combined_sim >= self.feature_similarity_threshold and
                deep_sim >= 0.6 and  # Strong deep feature requirement
                (iou >= self.iou_threshold or position_distance < self.position_threshold)):

                best_matches.append((player_id, final_score, combined_sim, iou, position_distance))

        # ==========================================
        # SELECT BEST MATCH
        # ==========================================
        # Choose the best matching player from candidates
        if best_matches:
            best_matches.sort(key=lambda x: x[1], reverse=True)
            best_match = best_matches[0]
            player_id, score, sim, iou, pos_dist = best_match

            # Additional verification for high confidence assignment
            if score >= self.confidence_threshold:
                self.tracker_to_player[tracker_id] = player_id
                self.update_player_data(player_id, box, features)
                # Increase confidence for successful matches
                self.player_confidence[player_id] = min(1.0, self.player_confidence.get(player_id, 0) + 0.1)

                # Set cooldown to prevent rapid switching
                self.id_switch_cooldown[player_id] = frame_count

                return player_id

        # ==========================================
        # ASSIGN NEW PLAYER ID
        # ==========================================
        # If no good match found and within player limit, create new player
        if self.next_player_id <= self.max_players:
            player_id = self.next_player_id
            self.next_player_id += 1

            self.tracker_to_player[tracker_id] = player_id
            self.initialize_player_data(player_id, box, features)
            self.player_confidence[player_id] = 0.5  # Initial confidence

            return player_id

        # No assignment possible (reached player limit or no good match)
        return None

    def update_player_data(self, player_id, box, features):
        """
        UPDATE PLAYER DATA STRUCTURES
        ============================
        Maintains historical data for each player across frames
        This history is crucial for accurate matching and tracking
        """
        # Update feature history using deque for efficient memory management
        if player_id not in self.player_features:
            self.player_features[player_id] = deque(maxlen=self.feature_history_size)
        self.player_features[player_id].append(features)

        # Update bounding box history
        if player_id not in self.player_boxes:
            self.player_boxes[player_id] = deque(maxlen=self.box_history_size)
        self.player_boxes[player_id].append(box)

        # Update position history (using center points for motion analysis)
        if player_id not in self.player_positions:
            self.player_positions[player_id] = deque(maxlen=self.position_history_size)
        center = ((box[0] + box[2]) / 2, (box[1] + box[3]) / 2)
        self.player_positions[player_id].append(center)

    def initialize_player_data(self, player_id, box, features):
        """
        INITIALIZE NEW PLAYER DATA
        =========================
        Set up data structures for a newly detected player
        """
        self.player_features[player_id] = deque([features], maxlen=self.feature_history_size)
        self.player_boxes[player_id] = deque([box], maxlen=self.box_history_size)
        center = ((box[0] + box[2]) / 2, (box[1] + box[3]) / 2)
        self.player_positions[player_id] = deque([center], maxlen=self.position_history_size)

    def update_missing_trackers(self, current_tracker_ids, frame_count):
        """
        MISSING TRACKER MANAGEMENT
        =========================
        Handles cases where players temporarily disappear from detection
        This is crucial for maintaining ID consistency across occlusions
        """
        active_trackers = set(current_tracker_ids)

        # Update missing counts for existing missing trackers
        for tracker_id in list(self.missing_trackers.keys()):
            if tracker_id in active_trackers:
                # Tracker is back, remove from missing list
                del self.missing_trackers[tracker_id]
            else:
                # Still missing, increment counter
                self.missing_trackers[tracker_id] += 1

        # Add newly missing trackers
        for tracker_id in list(self.tracker_to_player.keys()):
            if tracker_id not in active_trackers and tracker_id not in self.missing_trackers:
                self.missing_trackers[tracker_id] = 1

        # Clean up trackers that have been missing too long
        to_remove = []
        for tracker_id, missing_count in self.missing_trackers.items():
            if missing_count > self.max_missing_frames:
                to_remove.append(tracker_id)

        # Remove long-missing trackers and reduce their confidence
        for tracker_id in to_remove:
            if tracker_id in self.tracker_to_player:
                player_id = self.tracker_to_player[tracker_id]
                # Reduce confidence for disappeared players
                if player_id in self.player_confidence:
                    self.player_confidence[player_id] = max(0, self.player_confidence[player_id] - 0.2)
                del self.tracker_to_player[tracker_id]
            del self.missing_trackers[tracker_id]

# ==========================================
# MAIN TRACKING LOOP INITIALIZATION
# ==========================================
# Initialize the ultra-accurate tracker and prepare for processing
player_tracker = UltraAccuratePlayerTracker(feature_extractor, device)
frame_count = 0

print("\n🎯 Starting ultra-accurate player tracking...")
print("🔍 Features: Deep ResNet50 + Color + Position Prediction")
print("⚡ Optimized for minimal ID switching")

# ==========================================
# MAIN PROCESSING LOOP
# ==========================================
# Process each frame of the video with comprehensive tracking
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # ==========================================
    # YOLO DETECTION AND TRACKING
    # ==========================================
    # Run YOLO with built-in ByteTrack tracker for initial detection and basic tracking
    results = model.track(source=frame, persist=True, verbose=False, tracker="bytetrack.yaml")
    annotated_frame = frame.copy()

    current_tracker_ids = []

    # ==========================================
    # PROCESS DETECTIONS
    # ==========================================
    # Extract detections and apply our enhanced tracking logic
    if results[0].boxes.id is not None:
        boxes = results[0].boxes.xyxy.cpu().numpy()
        tracker_ids = results[0].boxes.id.cpu().numpy()

        # ==========================================
        # FILTER FOR PLAYER DETECTIONS ONLY
        # ==========================================
        # Remove non-player detections (referees, balls, etc.)
        player_detections = []
        for box, tracker_id in zip(boxes, tracker_ids):
            if player_tracker.is_player_detection(box):
                player_detections.append((box, int(tracker_id)))

        current_tracker_ids = [tid for _, tid in player_detections]

        # ==========================================
        # PROCESS EACH PLAYER DETECTION
        # ==========================================
        for box, tracker_id in player_detections:
            box = box.astype(int)
            x1, y1, x2, y2 = box

            # ==========================================
            # EXTRACT COMPREHENSIVE FEATURES
            # ==========================================
            # Get multi-modal features for robust identification
            features = player_tracker.extract_multi_features(frame, box)

            # ==========================================
            # ASSIGN PLAYER ID
            # ==========================================
            # Use our ultra-accurate matching system
            player_id = player_tracker.assign_player_id(tracker_id, box, features, frame_count)

            # ==========================================
            # DRAW ENHANCED ANNOTATIONS
            # ==========================================
            # Color-coded visualization based on confidence levels
            if player_id is not None:
                confidence = player_tracker.player_confidence.get(player_id, 0)
                if confidence > 0.7:
                    color = (0, 255, 0)  # Green for high confidence
                elif confidence > 0.4:
                    color = (0, 255, 255)  # Yellow for medium confidence
                else:
                    color = (0, 165, 255)  # Orange for low confidence

                label = f"P{player_id}"
                conf_text = f"{confidence:.2f}"
            else:
                color = (0, 0, 255)  # Red for unassigned
                label = "?"
                conf_text = "0.00"

            # Draw bounding box
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)

            # Draw player label
            cv2.putText(annotated_frame, label, (x1, y1-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

            # Draw confidence score
            cv2.putText(annotated_frame, conf_text, (x1, y2+20),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # ==========================================
    # UPDATE MISSING TRACKERS
    # ==========================================
    # Handle players that disappeared from current frame
    player_tracker.update_missing_trackers(current_tracker_ids, frame_count)

    # ==========================================
    # ENHANCED FRAME INFORMATION
    # ==========================================
    # Display comprehensive tracking statistics
    active_players = len(current_tracker_ids)
    total_players = player_tracker.next_player_id - 1
    info_text = f"Frame: {frame_count} | Active: {active_players} | Total: {total_players}"
    cv2.putText(annotated_frame, info_text, (10, 30),
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # ==========================================
    # OUTPUT GENERATION
    # ==========================================
    # Write frame to output video
    out.write(annotated_frame)

    # Save individual frame for analysis
    frame_filename = os.path.join(frames_dir, f"frame_{frame_count:04d}.jpg")
    cv2.imwrite(frame_filename, annotated_frame)

    # ==========================================
    # PROGRESS REPORTING
    # ==========================================
    # Regular progress updates every 30 frames
    if frame_count % 30 == 0:
        print(f"📊 Frame {frame_count}: {active_players} players active, {total_players} total detected")

# ==========================================
# CLEANUP AND FINAL RESULTS
# ==========================================
# Release resources and display final statistics
cap.release()
out.release()

print(f"\n✅ Ultra-accurate tracking complete!")
print(f"📁 Output video: {output_video_path}")
print(f"📁 Frames saved to: {frames_dir}")
print(f"🎯 Total players: {player_tracker.next_player_id - 1}")
print(f"📈 Final confidence scores: {player_tracker.player_confidence}")
print(f"🔄 ID mapping: {player_tracker.tracker_to_player}")


# ==========================================
# BASIC OUTPUT VERSION (COMMENTED)
# ==========================================
# This is a simplified version for basic detection without advanced tracking
# Uncomment this section if you want simple detection-only output

# from ultralytics import YOLO
# import cv2
# import os

# # ==========================================
# # BASIC DETECTION-ONLY VERSION
# # ==========================================
# # This simplified version performs only detection without advanced tracking
# # Useful for initial testing or when advanced tracking is not required

# # === File paths ===
# model_path = "/content/drive/MyDrive/Player_Tracking/Models/best.pt"
# video_path = "/content/drive/MyDrive/Player_Tracking/Input_Video/15sec_input_720p.mp4"
# output_dir = "/content/drive/MyDrive/Player_Tracking/Outputs"
# frames_dir = os.path.join(output_dir, "basic_tracked_frames")

# # === Create directories ===
# # Ensure output directories exist for saving results
# os.makedirs(output_dir, exist_ok=True)
# os.makedirs(frames_dir, exist_ok=True)

# # === Load YOLO model ===
# # Load pre-trained YOLO model for basic detection
# model = YOLO(model_path)
# print(f"✅ Loaded model from: {model_path}")

# # === Load video ===
# # Open input video file for processing
# cap = cv2.VideoCapture(video_path)
# if not cap.isOpened():
#     print("❌ Error opening video file.")
#     exit()

# # Get video properties for output configuration
# fps = cap.get(cv2.CAP_PROP_FPS)
# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# # === Set up video writer ===
# # Configure output video with same properties as input
# output_video_path = os.path.join(output_dir, "basic_tracked_output.avi")
# fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Video codec
# out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# # === Process frames ===
# # Main processing loop for basic detection
# frame_id = 0
# print("🚀 Starting detection-only run...")
# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # 👉 Run detection (no tracking)
#     # This performs only object detection without maintaining identities
#     results = model(frame)

#     # 👉 Annotate detections on the frame
#     # Draw bounding boxes and labels on detected objects
#     annotated_frame = results[0].plot()

#     # Optional: resize to original dimensions if needed
#     annotated_frame = cv2.resize(annotated_frame, (width, height))

#     # 👉 Write to video + save individual frames
#     # Save both video output and individual frames
#     out.write(annotated_frame)
#     frame_path = os.path.join(frames_dir, f"frame_{frame_id:04d}.jpg")
#     cv2.imwrite(frame_path, annotated_frame)

#     print(f"✅ Processed frame {frame_id}")
#     frame_id += 1

# # === Cleanup ===
# # Release video capture and writer resources
# cap.release()
# out.release()

# print(f"\n🎉 Detection complete.")
# print(f"📁 Output video saved to: {output_video_path}")
# print(f"📁 Frames saved in: {frames_dir}")

End of Code.

1. The Output video is In Outputs Folder named ultra_tracking.avi
2. Frame by Frame output is saved in folder tracked_frames in output.